In [19]:
import antimatter as am
import utils
import pandas as pd

amr = utils.create_or_load_domain()

amr.add_write_context(
    "data_science", am.WriteContextBuilder().
    set_summary("Data prepared for our DS team").
    # tag data based on the column names, for structured data
    add_hook("data-structure-classifier", ">1.0.0", am.WriteContextHookMode.Sync)
)

amr.add_read_context(
    "fraud_team", am.ReadContextBuilder().
    set_summary("Data access for fraud team").
    add_required_hook("data-structure-classifier", ">1.0.0")
)
amr.add_read_context(
    "ai_team", am.ReadContextBuilder().
    set_summary("Data access for AI team").
    add_required_hook("data-structure-classifier", ">1.0.0")
)
# clear out the rules from previous runs of this notebook
amr.delete_read_context_rules("fraud_team")
amr.delete_read_context_rules("ai_team")

# Redact names columns when the AI team reads data
amr.add_read_context_rules("ai_team", rule_builder=
    am.ReadContextRuleBuilder().add_match_expression(
        source=am.Source.Tags,
        key="tag.antimatter.io/key",
        operator=am.Operator.In,
        values=["first_name","last_name"]
    ).set_action(am.Action.Redact)
);


In [20]:
# Encapsulate our data
# we are doing a little extra, taking the tenant_id column and putting it into a tenant tag, so we can define governance policies 
# around the owner of the data
df = pd.read_parquet("mt-testdata.parquet")
df

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer4,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,Customer feedback: I recently visited your sto...
2,customer3,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,"Booking Confirmation: Thank you, David Smith (..."
3,customer1,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
4,customer2,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,nan,,"Urgent: My name is Sarah Lee, my SSN is 512-34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,customer2,196,Lillian,Perry,lperry5f@bigcartel.com,Female,35.71.128.205,,Portugal,,195580.83,,"Hello, I'm Elizabeth Anderson, age 29, and my ..."
196,customer2,197,Wanda,Allen,wallen5g@globo.com,Female,70.97.102.253,,Japan,,66158.0,,"Hi, I'm Paul Rivera, 45 years old, and I resid..."
197,customer2,198,Pamela,Tucker,ptucker5h@dedecms.com,Female,179.49.192.205,,Iran,10/23/1998,76690.52,Accountant III,"Hey there, I am Brian Powell, 35, living at 67..."
198,customer4,199,Aaron,Torres,atorres5i@w3.org,Male,37.19.3.217,677135008088348640,Canada,4/18/1980,85100.7,Sales Associate,"Hello, I am Angela Hall, 52 years old, staying..."


In [21]:

# Make a list of tags containing the tenant ID
customer_tags = [
    am.SpanTag(name="tenant", cell_path=am.cell_path("tenant_id", idx), tag_value=tenant, tag_type=am.cap_prep.tag.TagType.String,
               start=0, end=len(tenant)) 
    for (idx, tenant) in enumerate(df['tenant_id'])
]
# Encapsulate our data, passing in our custom customer tags
amr.encapsulate(path="testdata.cap", data_file_path="mt-testdata.parquet", span_tags=customer_tags,
                write_context="data_science")

In [22]:
# show what it's like for the fraud team to see the data
amr.load_capsule("testdata.cap", read_context="fraud_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer4,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,Customer feedback: I recently visited your sto...
2,customer3,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,"Booking Confirmation: Thank you, David Smith (..."
3,customer1,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
4,customer2,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,nan,,"Urgent: My name is Sarah Lee, my SSN is 512-34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,customer2,196,Lillian,Perry,lperry5f@bigcartel.com,Female,35.71.128.205,,Portugal,,195580.83,,"Hello, I'm Elizabeth Anderson, age 29, and my ..."
196,customer2,197,Wanda,Allen,wallen5g@globo.com,Female,70.97.102.253,,Japan,,66158.0,,"Hi, I'm Paul Rivera, 45 years old, and I resid..."
197,customer2,198,Pamela,Tucker,ptucker5h@dedecms.com,Female,179.49.192.205,,Iran,10/23/1998,76690.52,Accountant III,"Hey there, I am Brian Powell, 35, living at 67..."
198,customer4,199,Aaron,Torres,atorres5i@w3.org,Male,37.19.3.217,677135008088348640,Canada,4/18/1980,85100.7,Sales Associate,"Hello, I am Angela Hall, 52 years old, staying..."


In [23]:
# and for the AI team to see the data
amr.load_capsule("testdata.cap", read_context="ai_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,{redacted},{redacted},ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer4,2,{redacted},{redacted},afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,Customer feedback: I recently visited your sto...
2,customer3,3,{redacted},{redacted},emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,"Booking Confirmation: Thank you, David Smith (..."
3,customer1,4,{redacted},{redacted},driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
4,customer2,5,{redacted},{redacted},cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,nan,,"Urgent: My name is Sarah Lee, my SSN is 512-34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,customer2,196,{redacted},{redacted},lperry5f@bigcartel.com,Female,35.71.128.205,,Portugal,,195580.83,,"Hello, I'm Elizabeth Anderson, age 29, and my ..."
196,customer2,197,{redacted},{redacted},wallen5g@globo.com,Female,70.97.102.253,,Japan,,66158.0,,"Hi, I'm Paul Rivera, 45 years old, and I resid..."
197,customer2,198,{redacted},{redacted},ptucker5h@dedecms.com,Female,179.49.192.205,,Iran,10/23/1998,76690.52,Accountant III,"Hey there, I am Brian Powell, 35, living at 67..."
198,customer4,199,{redacted},{redacted},atorres5i@w3.org,Male,37.19.3.217,677135008088348640,Canada,4/18/1980,85100.7,Sales Associate,"Hello, I am Angela Hall, 52 years old, staying..."


## Customer opt-in

But now, let's say that the AI team can only read data if that customer has opted in to that data being used for AI:

In [24]:
# create a fact to store customer opt-in
amr.add_fact_type("customer_ai_opt_in", 
                  description="Stores customer opt-in for AI features",
                  arguments={
                      "customer": "the ID of the customer"
                  })

# lets say customer 1 and 3 have opted in
amr.add_fact("customer_ai_opt_in", "customer1")
amr.add_fact("customer_ai_opt_in", "customer3")

# Now let's also define a rule for the AI team read context that says to omit 
# data from customers that have not opted in
amr.add_read_context_rules("ai_team", rule_builder=
    am.ReadContextRuleBuilder().
       add_fact(am.Operator.NotExists, "customer_ai_opt_in",
                am.ReadContextRuleFactArgumentBuilder().
                add_argument(source=am.Source.Tags, key="tenant")
                ).
        set_action(am.Action.DenyRecord)
);

In [25]:
# Let's try read the data again:
amr.load_capsule("testdata.cap", read_context="ai_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,{redacted},{redacted},ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer3,3,{redacted},{redacted},emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,"Booking Confirmation: Thank you, David Smith (..."
2,customer1,4,{redacted},{redacted},driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
3,customer3,6,{redacted},{redacted},kwhite5@google.com,Female,195.131.81.179,3583136326049310,Indonesia,2/25/1983,69227.11,Account Executive,"Hello, I'm Mark Thompson. I’m 36 years old, re..."
4,customer1,7,{redacted},{redacted},sholmes6@foxnews.com,Male,232.234.81.197,3582641366974690,Portugal,12/18/1987,14247.62,Senior Financial Analyst,"Hi, my name is Michael Martinez, I am 40 years..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,customer3,190,{redacted},{redacted},rbishop59@deviantart.com,Male,108.100.129.98,,Russia,5/3/1958,75942.64,Desktop Support Technician,"Greetings! I am Daniel Carter, 38, and I live ..."
112,customer1,191,{redacted},{redacted},jwebb5a@dyndns.org,Female,169.75.241.122,3530639501736548,China,3/29/1979,127088.79,Speech Pathologist,"Hi, I am Laura Cook, 29 years old, and I live ..."
113,customer3,193,{redacted},{redacted},janderson5c@cbslocal.com,Female,157.218.30.218,4017951658384,Russia,12/12/1983,51701.32,Help Desk Operator,"Hello, I am Jessica Allen, age 33, staying at ..."
114,customer1,194,{redacted},{redacted},rgrant5d@skype.com,Male,206.151.211.209,5285674648398904,China,3/2/1990,70398.55,Senior Cost Accountant,"Hi, I'm Andrew Perez, 41 years old, residing a..."


In [26]:
# Now let's say customer 3 opts out:
amr.delete_fact(fact_type="customer_ai_opt_in", arguments=["customer3"])

In [27]:
# Let's try read the data again:
amr.load_capsule("testdata.cap", read_context="ai_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,{redacted},{redacted},ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer1,4,{redacted},{redacted},driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
2,customer1,7,{redacted},{redacted},sholmes6@foxnews.com,Male,232.234.81.197,3582641366974690,Portugal,12/18/1987,14247.62,Senior Financial Analyst,"Hi, my name is Michael Martinez, I am 40 years..."
3,customer1,9,{redacted},{redacted},jfoster8@yelp.com,Male,132.31.53.61,,South Korea,3/27/1992,231067.84,Software Test Engineer I,"Hey, it’s Lisa Davis, I am 28 years old. I not..."
4,customer1,12,{redacted},{redacted},aberryb@wikipedia.org,Female,246.225.12.189,4917830851454417,China,8/12/1968,22944.53,Quality Engineer,"Hi, I am Steven Lewis, aged 41, and I recently..."
5,customer1,17,{redacted},{redacted},eandrewsg@cornell.edu,Female,29.231.180.172,30271790537626,Russia,4/13/1990,116800.65,Food Chemist,"My name is Mary Lee, SSN 321-54-6789. I forgot..."
6,customer1,21,{redacted},{redacted},dstevensk@cnet.com,Female,141.243.73.164,,Russia,6/5/1985,87978.22,Food Chemist,"Hi, I'm Laura Walker, aged 33. I'm looking for..."
7,customer1,28,{redacted},{redacted},shunterr@instagram.com,Male,72.190.230.173,5002353797389897,Brazil,9/21/1968,108950.24,Environmental Tech,"Greetings, I'm Amanda Evans, aged 51. I reside..."
8,customer1,30,{redacted},{redacted},atorrest@ning.com,Female,202.94.67.27,3530389861801215,Nigeria,5/20/1958,118310.72,Electrical Engineer,"Hello, I'm Dorothy Coleman, age 57. I live at ..."
9,customer1,31,{redacted},{redacted},aberryu@ow.ly,Male,5.82.180.4,,Thailand,,135007.96,,"Hi, I'm Sandra Reed, 28 years old, staying at ..."


In [28]:
# But, as before, the fraud team continues to see all the data
amr.load_capsule("testdata.cap", read_context="fraud_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,Amanda,Jordan,ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer4,2,Albert,Freeman,afreeman1@is.gd,Male,218.111.175.34,,Canada,1/16/1968,150280.17,Accountant IV,Customer feedback: I recently visited your sto...
2,customer3,3,Evelyn,Morgan,emorgan2@altervista.org,Female,7.161.136.94,6767119071901597,Russia,2/1/1960,144972.51,Structural Engineer,"Booking Confirmation: Thank you, David Smith (..."
3,customer1,4,Denise,Riley,driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
4,customer2,5,Carlos,Burns,cburns4@miitbeian.gov.cn,,169.113.235.40,5602256255204850,South Africa,,nan,,"Urgent: My name is Sarah Lee, my SSN is 512-34..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,customer2,196,Lillian,Perry,lperry5f@bigcartel.com,Female,35.71.128.205,,Portugal,,195580.83,,"Hello, I'm Elizabeth Anderson, age 29, and my ..."
196,customer2,197,Wanda,Allen,wallen5g@globo.com,Female,70.97.102.253,,Japan,,66158.0,,"Hi, I'm Paul Rivera, 45 years old, and I resid..."
197,customer2,198,Pamela,Tucker,ptucker5h@dedecms.com,Female,179.49.192.205,,Iran,10/23/1998,76690.52,Accountant III,"Hey there, I am Brian Powell, 35, living at 67..."
198,customer4,199,Aaron,Torres,atorres5i@w3.org,Male,37.19.3.217,677135008088348640,Canada,4/18/1980,85100.7,Sales Associate,"Hello, I am Angela Hall, 52 years old, staying..."


In [29]:
# We can also easily create credentials for the AI team to use that are unable to use the fraud team's read_context
# First create the capability and the policy rule for it
amr.put_capability("ai-team", summary="Access given to the AI team", description="Access given to the AI team")
amr.create_policy_rule(am.CapabilityRules().
                        with_rule(name="ai-team",operator=am.CapabilityOperator.Exists),
                       facts=am.FactPolicies(),
                       path="control/read-context/ai_team",
                       operation=am.Operation.Use,
                       result=am.Result.Allow,
                       priority=9)
# Now create an API key for the AI team:
new_account = amr.insert_identity_provider_principal(provider_name="apikey",
                                       capabilities={"ai-team":None}, 
                                       principal_type=am.PrincipalType.ApiKey,
                                       principal_value=None)

/Volumes/Workspace/labs/launch-tutorials/ve/lib/python3.11/site-packages/pydantic/main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `_LiteralGenericAlias` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [30]:
# Connect using this API key:
amr_ai = am.Session(domain=amr.domain_id, api_key=new_account["api_key"])

# Read data in the AI read context:
amr_ai.load_capsule("testdata.cap", read_context="ai_team").data()

,tenant_id,id,first_name,last_name,email,gender,ip_address,cc,country,birthdate,salary,title,comments
0,customer1,1,{redacted},{redacted},ajordan0@com.com,Female,1.197.201.2,6759521864920116,Indonesia,3/8/1971,49756.53,Internal Auditor,"Hello friends, my name is Alice Johnson and I ..."
1,customer1,4,{redacted},{redacted},driley3@gmpg.org,Female,140.35.109.83,3576031598965625,China,4/8/1997,90263.05,Senior Cost Accountant,"Hi, I am Emily Brown, aged 33, and I recently ..."
2,customer1,7,{redacted},{redacted},sholmes6@foxnews.com,Male,232.234.81.197,3582641366974690,Portugal,12/18/1987,14247.62,Senior Financial Analyst,"Hi, my name is Michael Martinez, I am 40 years..."
3,customer1,9,{redacted},{redacted},jfoster8@yelp.com,Male,132.31.53.61,,South Korea,3/27/1992,231067.84,Software Test Engineer I,"Hey, it’s Lisa Davis, I am 28 years old. I not..."
4,customer1,12,{redacted},{redacted},aberryb@wikipedia.org,Female,246.225.12.189,4917830851454417,China,8/12/1968,22944.53,Quality Engineer,"Hi, I am Steven Lewis, aged 41, and I recently..."
5,customer1,17,{redacted},{redacted},eandrewsg@cornell.edu,Female,29.231.180.172,30271790537626,Russia,4/13/1990,116800.65,Food Chemist,"My name is Mary Lee, SSN 321-54-6789. I forgot..."
6,customer1,21,{redacted},{redacted},dstevensk@cnet.com,Female,141.243.73.164,,Russia,6/5/1985,87978.22,Food Chemist,"Hi, I'm Laura Walker, aged 33. I'm looking for..."
7,customer1,28,{redacted},{redacted},shunterr@instagram.com,Male,72.190.230.173,5002353797389897,Brazil,9/21/1968,108950.24,Environmental Tech,"Greetings, I'm Amanda Evans, aged 51. I reside..."
8,customer1,30,{redacted},{redacted},atorrest@ning.com,Female,202.94.67.27,3530389861801215,Nigeria,5/20/1958,118310.72,Electrical Engineer,"Hello, I'm Dorothy Coleman, age 57. I live at ..."
9,customer1,31,{redacted},{redacted},aberryu@ow.ly,Male,5.82.180.4,,Thailand,,135007.96,,"Hi, I'm Sandra Reed, 28 years old, staying at ..."


In [31]:
# Try use the fraud team read_context
amr_ai.load_capsule("testdata.cap", read_context="fraud_team").data()

RuntimeError: CapsuleError("No capsules were unsealed (access denied?)")